<summary>
    <font size="2" color="orange"><b>1.1 Limpieza y Pre- procesamiento. </b></font>
</summary>

Grupo 01.

Para el grupo 1, se ajusta el formato de columnas manteniendo el resto de las variables y agregando como indice una columna con formato datetime, al final de esta libreta, se exporta el nuevo dataframe para

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


In [3]:
print(os.getcwd())

/Users/yhmve/AAA2023/AAA2023/Grupo_01


Base de datos original contiene
+ columna fecha
+ Demanda de energia historica (columnas H1 a H24)
+ Temperaturas Max HMO, Los Mochis, Culican, Caborca
+ Temperaturs MInimas HMO, LOs Mochis, Culiacan, Caborca
+ Precipitacion HMO, Obregon, Mochis, Culiacan
+ Lunes festivos, Martes Posfestivos, Y Festivos en Mexico 
+ Caso especial Navidad Pre-Navidad y Post Navidad y Año nuevo





In [4]:
# Importing load energy consumption CENACE database
#gcrno = pd.read_csv('../Grupo_01/data1/DATASETGCRNODIARIO20230118.csv', delimiter=",")
df = pd.read_csv('../Grupo_01/q2/DATASETGCRNODIARIO20230329.csv', delimiter=",")
df.columns

Index(['FECHA', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10',
       'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20',
       'H21', 'H22', 'H23', 'H24', 'TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR',
       'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB', 'TMIN-HMO', 'TMIN-OBR', 'TMIN-LMO',
       'TMIN-CUL', 'PREC_HMO_MM', 'PREC_OBR_MM', 'PREC_LMO_MM', 'PREC_CUL_MM',
       'LUNES_FESTIVO', 'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO',
       '10_MAYO', '16_SEP', '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR',
       'NAVIDAD_Y_NEW_YEAR', 'POST-NAVIDAD_Y_NEW_YEAR'],
      dtype='object')

In [5]:
# rename the columns using a dictionary  to adjust H0 to H23
new_names = {f'H{i}': f'H{i-1}' for i in range(1, 25)}
gcrno = df.rename(columns=new_names)
gcrno.columns

Index(['FECHA', 'H0', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9',
       'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19',
       'H20', 'H21', 'H22', 'H23', 'TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR',
       'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB', 'TMIN-HMO', 'TMIN-OBR', 'TMIN-LMO',
       'TMIN-CUL', 'PREC_HMO_MM', 'PREC_OBR_MM', 'PREC_LMO_MM', 'PREC_CUL_MM',
       'LUNES_FESTIVO', 'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO',
       '10_MAYO', '16_SEP', '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR',
       'NAVIDAD_Y_NEW_YEAR', 'POST-NAVIDAD_Y_NEW_YEAR'],
      dtype='object')

In [6]:
#set working directory  
#os.chdir('./data1')
#print(os.getcwd())

### Ajustes de Dataframe

Dataframe incluye historicos de 2007/01/01 to 2023/03/29 <br>
Se ajustaran las columnas de H1 ~ H24 para que queden contenidas en una sola columna como Fecha Hora<br>
Se ajustan nombres de columnas a snake case<br>

In [7]:
# Transposing hours columns from the original dataframe into rows

consumption = gcrno.melt(
    id_vars= ['FECHA'],
    value_vars= [f'H{i}' for i in range(24)],
    var_name="HORA",
    value_name="DEMANDA"
).replace(
    {f'H{i}': i for i in range(24)}
    )

In [8]:
# Creating Day, Hour and Month columns
consumption['FECHA']= pd.to_datetime(consumption['FECHA'], format='%d/%m/%Y')
consumption.index = consumption.FECHA + pd.to_timedelta(consumption.HORA, unit='h')
consumption.sort_index(inplace=True)
consumption.drop(columns=['HORA'], inplace=True)
consumption = consumption.asfreq('h', method='pad')
consumption['Date_time'] = consumption.index
consumption["Day"] = consumption.index.weekday
consumption["Hour"] = consumption.index.hour
consumption["Month"] = consumption.index.month
consumption["Year"] = consumption.index.year


In [9]:
#updating column titles in consumo
consumption.rename(columns={"FECHA":"Date",'DEMANDA':'Energy_Demand'}, inplace = True)

Las varias exógenas son factores externos del modelo, en este caso se refiere a las variables de temperatura, precipitacion y festivos.


In [10]:
# Adding columns of exogenous variables
exogenous = gcrno[['FECHA', 'TMAX-CAB', 'TMAX-HMO', 'TMAX-OBR',
       'TMAX-LMO', 'TMAX-CUL', 'TMIN-CAB', 'TMIN-HMO', 'TMIN-OBR', 
       'TMIN-LMO','TMIN-CUL',  'PREC_HMO_MM','PREC_OBR_MM', 
       'PREC_LMO_MM', 'PREC_CUL_MM', 'LUNES_FESTIVO',
       'MARTES_POSTFESTIVO', 'SEMANA_SANTA', '1_MAYO', '10_MAYO', '16_SEP',
       '2_NOV.', 'PRE-NAVIDAD_Y_NEW_YEAR', 'NAVIDAD_Y_NEW_YEAR',
       'POST-NAVIDAD_Y_NEW_YEAR']]

In [11]:
#updating column names to title case 
exogenous.columns = [col.title() for col in exogenous.columns]
exogenous.head(2)

,Fecha,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,Tmin-Hmo,Tmin-Obr,Tmin-Lmo,...,Lunes_Festivo,Martes_Postfestivo,Semana_Santa,1_Mayo,10_Mayo,16_Sep,2_Nov.,Pre-Navidad_Y_New_Year,Navidad_Y_New_Year,Post-Navidad_Y_New_Year
0,01/01/2007,21.0,22.0,25.0,30.0,29.0,2.0,9.0,8.0,10.0,...,0,0,0,0,0,0,0,0,1,0
1,02/01/2007,21.0,22.0,22.0,22.0,27.0,2.0,7.0,7.0,11.0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
exogenous.rename(columns={"Fecha":"Date",'Lunes_Festivo':'Monday_Holiday',
       'Martes_Postfestivo':'Tuesday_Aft_Hol', 'Semana_Santa':'Easter_week', '1_Mayo':'May_1s', 
       '10_Mayo':'May_10t', '16_Sep':'Sept_16','2_Nov.':'Nov_2nd',
       'Pre-Navidad_Y_New_Year':'Before_Christmas_NY', 'Navidad_Y_New_Year':'Christmas_NY',
       'Post-Navidad_Y_New_Year':'After_Christmas_NY'}, inplace = True)

In [13]:
consumption.columns

Index(['Date', 'Energy_Demand', 'Date_time', 'Day', 'Hour', 'Month', 'Year'], dtype='object')

In [14]:
exogenous['Date']= pd.to_datetime(exogenous['Date'], format='%d/%m/%Y')
consumption_com = pd.merge(consumption, exogenous, on='Date', how='left')

In [15]:
consumption_com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142368 entries, 0 to 142367
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date                 142368 non-null  datetime64[ns]
 1   Energy_Demand        142368 non-null  float64       
 2   Date_time            142368 non-null  datetime64[ns]
 3   Day                  142368 non-null  int64         
 4   Hour                 142368 non-null  int64         
 5   Month                142368 non-null  int64         
 6   Year                 142368 non-null  int64         
 7   Tmax-Cab             142368 non-null  float64       
 8   Tmax-Hmo             142368 non-null  float64       
 9   Tmax-Obr             142368 non-null  float64       
 10  Tmax-Lmo             142368 non-null  float64       
 11  Tmax-Cul             142368 non-null  float64       
 12  Tmin-Cab             142368 non-null  float64       
 13  Tmin-Hmo      

In [16]:
# Setting as index the Date_time
consumption_com.set_index("Date_time", inplace=True)
consumption_com=consumption_com.asfreq('h')

In [17]:
consumption_com.tail(3)

,Date,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
Date_time,,,,,,,,,,,,,,,,,,,,,
2023-03-29 21:00:00,2023-03-29,2654.24,2,21,3,2023,30.2,32.0,32.1,31.5,...,0,0,0,0,0,0,0,0,0,0
2023-03-29 22:00:00,2023-03-29,2584.34,2,22,3,2023,30.2,32.0,32.1,31.5,...,0,0,0,0,0,0,0,0,0,0
2023-03-29 23:00:00,2023-03-29,2492.83,2,23,3,2023,30.2,32.0,32.1,31.5,...,0,0,0,0,0,0,0,0,0,0


In [18]:
consumption_com['Day'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [19]:
# Verifying existence of missing data 
consumption_com.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 142368 entries, 2007-01-01 00:00:00 to 2023-03-29 23:00:00
Freq: H
Data columns (total 30 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Date                 142368 non-null  datetime64[ns]
 1   Energy_Demand        142368 non-null  float64       
 2   Day                  142368 non-null  int64         
 3   Hour                 142368 non-null  int64         
 4   Month                142368 non-null  int64         
 5   Year                 142368 non-null  int64         
 6   Tmax-Cab             142368 non-null  float64       
 7   Tmax-Hmo             142368 non-null  float64       
 8   Tmax-Obr             142368 non-null  float64       
 9   Tmax-Lmo             142368 non-null  float64       
 10  Tmax-Cul             142368 non-null  float64       
 11  Tmin-Cab             142368 non-null  float64       
 12  Tmin-Hmo             142368 no

<summary>
    <font size="4" color="orange"><b>2. Exploring variables</b></font>
</summary>

<br/>
<summary>
    <font size="3" color="palevioletred"><b>Energy Demand</b></font>
</summary>

In [20]:
#looking for more information on endogenous Features Demanda
consumption_com['Energy_Demand'].describe()

count    142368.000000
mean       2497.023214
std         814.677130
min         959.000000
25%        1875.000000
50%        2336.610000
75%        3027.165000
max        5402.720000
Name: Energy_Demand, dtype: float64

<summary>
    <font size="3" color="palevioletred"><b>Exogenous Meteorological Features</b></font>
</summary>

In [21]:
consumption_com.head(2)

,Date,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
Date_time,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,2007-01-01,1297.0,0,0,1,2007,21.0,22.0,25.0,30.0,...,0,0,0,0,0,0,0,0,1,0
2007-01-01 01:00:00,2007-01-01,1255.0,0,1,1,2007,21.0,22.0,25.0,30.0,...,0,0,0,0,0,0,0,0,1,0


In [22]:
#looking for more information on enxogenous Features Tmax & Tmin
consumption_com[['Tmax-Cab', 'Tmax-Hmo',
       'Tmax-Obr', 'Tmax-Lmo', 'Tmax-Cul', 'Tmin-Cab', 'Tmin-Hmo', 'Tmin-Obr',
       'Tmin-Lmo', 'Tmin-Cul']].describe()

,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,Tmin-Hmo,Tmin-Obr,Tmin-Lmo,Tmin-Cul
count,142368.000000,142368.000000,142368.000000,142368.000000,142368.000000,142368.000000,142368.000000,142368.000000,142368.000000,142368.000000
mean,32.734127,33.423763,34.349742,32.278102,33.849801,16.256816,18.324481,18.421483,18.863675,20.157264
std,7.907491,6.678481,5.998366,4.902231,4.095490,7.984333,7.106331,6.846368,6.137698,5.498096
min,9.000000,8.000000,12.000000,12.000000,17.000000,-7.000000,-3.000000,2.000000,1.000000,1.000000
25%,26.400000,28.345000,30.000000,28.657500,31.000000,10.000000,12.510000,13.000000,13.500000,16.000000
50%,33.200000,34.000000,35.000000,33.000000,34.000000,16.000000,18.000000,18.000000,18.000000,20.000000
75%,39.200000,39.000000,39.000000,36.000000,37.000000,23.100000,25.000000,25.000000,25.000000,25.000000
max,50.000000,49.100000,47.000000,45.000000,44.000000,33.000000,34.000000,42.500000,37.000000,32.000000


In [23]:
#To export file remote  "date"
consumption_com.drop('Date', inplace=True, axis=1)
consumption_com.head(2)

,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,...,Monday_Holiday,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY
Date_time,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,1297.0,0,0,1,2007,21.0,22.0,25.0,30.0,29.0,...,0,0,0,0,0,0,0,0,1,0
2007-01-01 01:00:00,1255.0,0,1,1,2007,21.0,22.0,25.0,30.0,29.0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
print(os.getcwd())

/Users/yhmve/energy_demand_anlys/Grupo_01


In [25]:
os.chdir('./q2')
print(os.getcwd())

/Users/yhmve/energy_demand_anlys/Grupo_01/q2


In [26]:
#Export file
#send combined dataframe to file
#consumption_com.to_csv (r'group01_allq2.csv', index = True)

In [27]:
#Create dataframe with actual values to compare with Forecasted metrics
consumption_com.columns

Index(['Energy_Demand', 'Day', 'Hour', 'Month', 'Year', 'Tmax-Cab', 'Tmax-Hmo',
       'Tmax-Obr', 'Tmax-Lmo', 'Tmax-Cul', 'Tmin-Cab', 'Tmin-Hmo', 'Tmin-Obr',
       'Tmin-Lmo', 'Tmin-Cul', 'Prec_Hmo_Mm', 'Prec_Obr_Mm', 'Prec_Lmo_Mm',
       'Prec_Cul_Mm', 'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week',
       'May_1s', 'May_10t', 'Sept_16', 'Nov_2nd', 'Before_Christmas_NY',
       'Christmas_NY', 'After_Christmas_NY'],
      dtype='object')

In [32]:
actualcols = ['Energy_Demand', 'Day', 'Hour', 'Month', 'Year', 'Tmax-Cab', 'Tmax-Hmo',
       'Tmax-Obr', 'Tmax-Lmo', 'Tmax-Cul', 'Tmin-Cab', 'Tmin-Hmo', 'Tmin-Obr',
       'Tmin-Lmo', 'Tmin-Cul', 'Prec_Hmo_Mm', 'Prec_Obr_Mm', 'Prec_Lmo_Mm',
       'Prec_Cul_Mm']
df_actuals = consumption_com[actualcols]
df_actuals.head(2)

,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,Tmin-Hmo,Tmin-Obr,Tmin-Lmo,Tmin-Cul,Prec_Hmo_Mm,Prec_Obr_Mm,Prec_Lmo_Mm,Prec_Cul_Mm
Date_time,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,1297.0,0,0,1,2007,21.0,22.0,25.0,30.0,29.0,2.0,9.0,8.0,10.0,9.0,0.0,0.0,0.0,0.0
2007-01-01 01:00:00,1255.0,0,1,1,2007,21.0,22.0,25.0,30.0,29.0,2.0,9.0,8.0,10.0,9.0,0.0,0.0,0.0,0.0


In [29]:
df_actuals.tail(3)

,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,Tmin-Hmo,Tmin-Obr,Tmin-Lmo,Tmin-Cul,Prec_Hmo_Mm,Prec_Obr_Mm,Prec_Lmo_Mm,Prec_Cul_Mm
Date_time,,,,,,,,,,,,,,,,,,,
2023-03-29 21:00:00,2654.24,2,21,3,2023,30.2,32.0,32.1,31.5,33.0,12.4,13.5,11.0,14.5,16.0,0.0,0.0,0.0,0.0
2023-03-29 22:00:00,2584.34,2,22,3,2023,30.2,32.0,32.1,31.5,33.0,12.4,13.5,11.0,14.5,16.0,0.0,0.0,0.0,0.0
2023-03-29 23:00:00,2492.83,2,23,3,2023,30.2,32.0,32.1,31.5,33.0,12.4,13.5,11.0,14.5,16.0,0.0,0.0,0.0,0.0


TO check with group 4 Data needs to select data from  01/04/2021 to 30/03/2023

In [33]:
df_actuals = df_actuals.loc['2021-04-01':'2023-03-31']

In [35]:
df_actuals.tail(2)

,Energy_Demand,Day,Hour,Month,Year,Tmax-Cab,Tmax-Hmo,Tmax-Obr,Tmax-Lmo,Tmax-Cul,Tmin-Cab,Tmin-Hmo,Tmin-Obr,Tmin-Lmo,Tmin-Cul,Prec_Hmo_Mm,Prec_Obr_Mm,Prec_Lmo_Mm,Prec_Cul_Mm
Date_time,,,,,,,,,,,,,,,,,,,
2023-03-29 22:00:00,2584.34,2,22,3,2023,30.2,32.0,32.1,31.5,33.0,12.4,13.5,11.0,14.5,16.0,0.0,0.0,0.0,0.0
2023-03-29 23:00:00,2492.83,2,23,3,2023,30.2,32.0,32.1,31.5,33.0,12.4,13.5,11.0,14.5,16.0,0.0,0.0,0.0,0.0


In [49]:
os.chdir("../..")#change directory 

In [53]:
#Export file
#send combined dataframe to file
df_actuals.to_csv (r'./Grupo_4/q2/actualsvars_q2.csv', index = True)

In [54]:
print(os.getcwd())

/Users/yhmve/energy_demand_anlys
